In [1]:
import wget
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
pdf_url = 'https://ncrb.gov.in/sites/default/files/adsi_reports_previous_year/Table-1.11_2019.pdf'
fileName = pdf_url[67:]
try:
    #Delete the file to redownload it
    os.remove(fileName)
except FileNotFoundError as e:
    #Ignore in case FileNotFound Error occurs
    pass
fileName = wget.download(pdf_url,fileName)

In [3]:
import numpy as np
import pandas as pd
import camelot

In [4]:
fileName

'Table-1.11_2019.pdf'

In [5]:
tables = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=["25.972342487883687,716.0485040267723,569.4676575121164,59.03236513239123"],
                          columns=["52.90662358642973,152.94823909531505,212.5884329563813,256.83760904684976,301.0867851373183,342.4501453957997,384.7754442649435,430.9484975767367,477.12155088852995,515.5990953150243"],
                          split_text=True,
                          pages="all"
                         )

In [6]:
tables

<TableList n=14>

In [7]:
def printnAllTables(tables):
    for i in range(len(tables)):
        print("Page No: ",(i+1))
        print(tables[i].df)
        print()

In [8]:
# Incase we're intersted in seeing the original scraped tables.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [9]:
# Removes Garbage Rows that may have been detected Before the 'STATES' row
# AND
# Removes Garbage Rows that may have been detected after the 'TOTAL (ALL INDIA)' row
def clean(table):
    while(table.df.iloc[0,1] != 'STATES'):
        table.df = table.df.iloc[1:]
    while(table.df.iloc[-1,1] != 'TOTAL (ALL INDIA)'):        table.df = table.df.iloc[:-1]
    # The Return line is unnecessary due to all changes being reflected in the original references
    return table

In [10]:
for each_table in tables:
    clean(each_table)

In [11]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [12]:
# List of Column headers as column are nested, i.e exhibit a multi level/index structure
# The multi level structre will be generated and applied in the next 
column_header = [
    "Fire in School Buildings",
    "Fire in Commercial Buildings",
    "Fire in Residential/Dwelling Buidings",
    "Fire in Government Buildings",
    "Fire in the Mines",
    "Fire in Factory Manufacturing Combustible Materials Including Cracker/Match Box Factories",
    "Fire in the Other Factories",
    "Accidental Fire in Train(s)",
    "Accidental Fire in Vehicles (Automobiles) [Total]",
    "Accidental Fire in Vehicles (Automobiles) [Passenger Vehicle (e.g. Bus/Taxi/Auto/etc.)]",
    "Accidental Fire in Vehicles (Automobiles) [Private Vehicle]",
    "Accidental Fire in Vehicles (Automobiles) [Goods Carriers]",
    "Others",
    "Total"
]
len(column_header)

14

In [13]:
def set_Column(df, title):
    column_list = [
        ("Sl. No.",'',''),
        ("State/UT",'',''), 
        (title,"No. of Cases",''),
        (title,'No. of Persons Injured','Male'),
        (title,'No. of Persons Injured','Female'),
        (title,'No. of Persons Injured','Transgender'),
        (title,'No. of Persons Injured','Total'),
        (title,'No. of Persons Died','Male'),
        (title,'No. of Persons Died','Female'),
        (title,'No. of Persons Died','Transgender'),
        (title,'No. of Persons Died','Total'),
    ]
    df.columns = pd.MultiIndex.from_tuples(column_list)

In [14]:
for i in range(len(tables)):
    set_Column(tables[i].df, column_header[i])
    tables[i].df.set_index(["Sl. No.","State/UT"],inplace=True)

In [15]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

# OR Print a simgle one according to your choice
# tables[23].df

In [16]:
# Convert List of tables to list of DataFrames
DataFrames = [each.df.copy() for each in tables]
# Concat all the tables one after the other, as they all have the same indexes.
final = pd.concat(DataFrames,axis=1)

In [17]:
# Reset Index before saving file, for better formatting in RAW CSV
final.reset_index().to_csv("Place_of_Occurrence-wise_Number_of_Fire_Accidents,_Persons_Injured_and_Died_during_2019_(State_&_UT-wise).csv",index=False)